In [156]:
#creating a dataframe
path="/home/affine/Expedia/Random Forest/AD_Region_Mar_1.csv"
AD_Region_Mar = sqlCtx.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',delimiter=',').load(path)

In [157]:
AD_Region_Mar.count()

40190

In [162]:
AD_Region_Mar.printSchema()

root
 |-- hotel_id: integer (nullable = true)
 |-- region: string (nullable = true)
 |-- region_bucket: string (nullable = true)
 |-- market_bucket: string (nullable = true)
 |-- market: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- city_bucket: string (nullable = true)
 |-- city: string (nullable = true)
 |-- hotel_type: string (nullable = true)
 |-- hotel_type_bucket: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- star_rating_bucket: string (nullable = true)
 |-- los_bucket: string (nullable = true)
 |-- bw_bucket: string (nullable = true)
 |-- device_type: string (nullable = true)
 |-- activity_date: string (nullable = true)
 |-- traffic: integer (nullable = true)
 |-- booking: integer (nullable = true)
 |-- total_cost: double (nullable = true)
 |-- gross_profit: double (nullable = true)
 |-- seller_rank: integer (nullable = true)
 |-- seller_rank_bucket: string (nullable = true)



In [166]:
from pyspark.sql.functions import *
import datetime
from pyspark.sql.types import StringType

def change_date(date):
    date = datetime.datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d')
    return date

change_date_udf = udf(change_date, StringType())
#AD_Region_Mar_new=AD_Region_Mar.withColumn("activity_date_new",change_date_udf(AD_Region_Mar.activity_date))

#AD_Region_Mar_train = AD_Region_Mar_new.filter((AD_Region_Mar_new["activity_date_new"] >= "2015-01-01") & (AD_Region_Mar_new["activity_date_new"] <= "2015-01-27"))
#AD_Region_Mar_val = AD_Region_Mar_new.filter((AD_Region_Mar_new["activity_date_new"] < "2015-01-01") | (AD_Region_Mar_new["activity_date_new"] > "2015-01-27"))

AD_Region_Mar_train = AD_Region_Mar
AD_Region_Mar_val = AD_Region_Mar


In [167]:
#Aggregating the data to the required level
from pyspark.sql.functions import *
AD_Region_Mar_T = AD_Region_Mar_train.groupBy(AD_Region_Mar_train.bw_bucket,AD_Region_Mar_train.los_bucket \
                      ,AD_Region_Mar_train.hotel_type_bucket,AD_Region_Mar_train.star_rating_bucket,AD_Region_Mar_train.market_bucket \
                      ,AD_Region_Mar_train.seller_rank_bucket).sum("traffic","booking")

AD_Region_Mar_T1 = AD_Region_Mar_T.withColumnRenamed('sum(traffic)','Traffic')
AD_Region_Mar_T1 = AD_Region_Mar_T1.withColumnRenamed('sum(booking)','Booking')


AD_Region_Mar_V = AD_Region_Mar_val.groupBy(AD_Region_Mar_val.bw_bucket,AD_Region_Mar_val.los_bucket \
                      ,AD_Region_Mar_val.hotel_type_bucket,AD_Region_Mar_val.star_rating_bucket,AD_Region_Mar_val.market_bucket \
                      ,AD_Region_Mar_val.seller_rank_bucket).sum("traffic","booking")

AD_Region_Mar_V1 = AD_Region_Mar_V.withColumnRenamed('sum(traffic)','Traffic')
AD_Region_Mar_V1 = AD_Region_Mar_V1.withColumnRenamed('sum(booking)','Booking')

######Filtering out the Traffic (Outliers) from data
AD_Region_Mar_T1 = AD_Region_Mar_T1[~((AD_Region_Mar_T1.Traffic<=4) & (AD_Region_Mar_T1.Booking>0))]
AD_Region_Mar_T1 = AD_Region_Mar_T1[~((AD_Region_Mar_T1.Traffic==5) & (AD_Region_Mar_T1.Booking==2))]
AD_Region_Mar_T1 = AD_Region_Mar_T1[~((AD_Region_Mar_T1.Traffic==6) & (AD_Region_Mar_T1.Booking==2))]


In [168]:
import numpy

#####Converting the data type of required columns to factor
from pyspark.ml.feature import *
def indexStringColumns(df,cols):
    tempdf = df
    for col in cols:
        stringIndexer = StringIndexer(inputCol=col,outputCol=col+"-num")
        si_model = stringIndexer.fit(tempdf)
        tempdf = si_model.transform(tempdf)
    return tempdf
cols = {"bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket"}
dfnumeric = indexStringColumns(AD_Region_Mar_T1,cols)

In [169]:
def oneHotEncodeColumns(df,cols):
    tempdf = df
    for col in cols:
        onehotenc = OneHotEncoder(inputCol=col+"-num",outputCol=col+"-onehot")
        tempdf = onehotenc.transform(tempdf).drop(col+"-num")
    return tempdf
AD_Region_Mar_T1_Class = oneHotEncodeColumns(dfnumeric,{"bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket"})

In [170]:
#####Transforming the training dataset
#Defining function for the same
def genRows(row):
    traffic = row[6]
    booking = row[7]
    bookingcounter = 0
    rowList = []
    
    for i in range(traffic):
        row1 = row[:]                   
        #row1[6]=1
        row1= list(row1)
        row1[6]=1
        row1 = tuple(row1)
        if bookingcounter < int(booking):
            #row1[7]=1
            row1= list(row1)
            row1[7]=1
            row1 = tuple(row1)
        else :
            #row1[7]=0
            row1= list(row1)
            row1[7]=0
            row1 = tuple(row1)
        bookingcounter += 1
        rowList.append(row1)
    return rowList

#Applying function on the complete dataset (after converting to rdd)
rdd1 = AD_Region_Mar_T1_Class.rdd
rdd2 = rdd1.flatMap(lambda x : genRows(x))

In [174]:
#Reconverting to dataframe
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.mllib.linalg import SparseVector, VectorUDT


schema = StructType([StructField('bw_bucket', StringType(), True),
                     StructField('los_bucket', StringType(), True),
                     StructField('hotel_type_bucket', StringType(), True),
                     StructField('star_rating_bucket', StringType(), True),
                     StructField('market_bucket', StringType(), True),
                     StructField('seller_rank_bucket', StringType(), True),
                     StructField('Traffic', IntegerType(), True),
                     StructField('Booking', IntegerType(), True),
                     StructField('hotel_type_bucket-onehot', VectorUDT(), True),
                     StructField('bw_bucket-onehot',  VectorUDT(), True),
                     StructField('region_bucket-onehot',  VectorUDT(), True),
                     StructField('los_bucket-onehot',  VectorUDT(), True),
                     StructField('seller_rank_bucket-onehot',  VectorUDT(), True),
                     StructField('star_rating_bucket-onehot',  VectorUDT(), True)])
df7 = sqlContext.createDataFrame(rdd2, schema)

In [175]:
from pyspark.sql.functions import udf

#Converting label to double datatype
toDouble = udf(lambda x : float(x * 1.00) , DoubleType())
df8 = df7.withColumn('Booking' , toDouble(df7.Booking))
#df8.dtypes

#Taking only relevant columns as features
colList = df8.columns 
colList.remove('Booking')
colList.remove('Traffic')
colList.remove('bw_bucket')
colList.remove('los_bucket')
colList.remove('hotel_type_bucket')
colList.remove('star_rating_bucket')
colList.remove('market_bucket')
colList.remove('seller_rank_bucket')
vecAssembler = VectorAssembler(inputCols=colList,outputCol="features")
Train_AD = vecAssembler.transform(df8).select("Booking","Traffic","bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket","features").withColumnRenamed("Booking", "label")


In [176]:
#####Transforming the validation dataset
cols = {"bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket"}
dfnumeric_V1 = indexStringColumns(AD_Region_Mar_T1,cols)
#dfnumeric_V1.show()
df9 = oneHotEncodeColumns(dfnumeric_V1,{"bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket"})

col = df9.columns 
col.remove('Booking')
col.remove('Traffic')
col.remove('bw_bucket')
col.remove('los_bucket')
col.remove('hotel_type_bucket')
col.remove('star_rating_bucket')
col.remove('market_bucket')
col.remove('seller_rank_bucket')
vecAssembler = VectorAssembler(inputCols=col,outputCol="indexedFeatures")
Valid_DF = vecAssembler.transform(df9).select("Booking","Traffic","bw_bucket","los_bucket","hotel_type_bucket","star_rating_bucket","market_bucket","seller_rank_bucket","indexedFeatures")



In [218]:
####Random Forest model building
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(Train_AD)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures").fit(Train_AD)
rf = RandomForestRegressor(labelCol="indexedLabel", featuresCol="indexedFeatures",featureSubsetStrategy="onethird",\
                           numTrees=1000,impurity="variance",maxBins=32)
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])
model = pipeline.fit(Train_AD)
rfModel = model.stages[2]
print (rfModel)

####Prediction on validation dataset
predictions = rfModel.transform(Valid_DF)
#predictions.printSchema()


RandomForestRegressionModel (uid=rfr_d80e1edd9d23) with 1000 trees


In [219]:

def actual_conv(traffic,booking):
    return (booking/traffic)

actual_conv_udf = udf (actual_conv, DoubleType())

predictions = predictions.withColumn("actual_conv",actual_conv_udf(predictions.Traffic,predictions.Booking))
   

In [220]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    labelCol="actual_conv", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

In [221]:
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.0397409


In [181]:
rfModel.featureImportances

AttributeError: 'RandomForestRegressionModel' object has no attribute 'featureImportances'

In [222]:
path="/home/affine/Expedia/Random Forest/RF_AD_Region_T1_pred_1.csv"
r_ad = sqlCtx.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',delimiter=',').load(path)
r_ad = r_ad.withColumnRenamed("traffic","Traffic").withColumnRenamed("booking","Booking")

In [223]:
df_final = r_ad.join(predictions,["bw_bucket","los_bucket","market_bucket","seller_rank_bucket","hotel_type_bucket","star_rating_bucket","Traffic","Booking"],"inner")

In [224]:
df_final = df_final.withColumnRenamed("Predicted","Predict_R").withColumnRenamed("prediction","Predict_Py")

In [225]:
df_final.describe('Predict_R','Predict_Py').show()

+-------+--------------------+--------------------+
|summary|           Predict_R|          Predict_Py|
+-------+--------------------+--------------------+
|  count|               11951|               11951|
|   mean|0.016114462591071965|0.024577660378729743|
| stddev|0.024699923504858542| 0.01799226731873051|
|    min|                 0.0|0.003466579230573777|
|    max|   0.307621776403908| 0.09580963988275441|
+-------+--------------------+--------------------+



In [226]:
df_temp = df_final

In [227]:
def error(actual,predict):
    x = actual-predict  
    
    if x < 0.0:
        x  = x*(-1)
    else : x
    
    if actual == 0.0:
        y = x
    else :
        y = x/actual
    return x

error_udf = udf(error,DoubleType())

df_final1 =  df_temp.withColumn("error_R",error_udf(df_temp.actual_conv,df_temp.Predict_R)).withColumn("error_Py",error_udf(df_temp.actual_conv,df_temp.Predict_Py))

In [228]:
def error_new(Predict_R,Predict_Py):
    x = Predict_R-Predict_Py  
    
    if x < 0.0:
        x  = x*(-1)
    else : x
    
    return x

error_new_udf = udf(error_new,DoubleType())

df_final1 = df_final1.withColumn("error_r_py",error_new_udf(df_final1.Predict_R,df_final1.Predict_Py))

In [229]:
df_final1.agg({"error_r_py":"avg","error_R":"avg","error_py":"avg"}).show()

+--------------------+--------------------+--------------------+
|     avg(error_r_py)|       avg(error_py)|        avg(error_R)|
+--------------------+--------------------+--------------------+
|0.015007416135886619|0.027142913051916462|0.012822558342755024|
+--------------------+--------------------+--------------------+



In [230]:
evaluator = RegressionEvaluator(
    labelCol="Predict_R", predictionCol="Predict_Py", metricName="rmse")
rmse = evaluator.evaluate(df_final1)
print(rmse)

0.02030657928645599


In [231]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
df_final1 = df_final1.withColumn("Booking_1",df_final1.Booking.cast(DoubleType()))


evaluator = BinaryClassificationEvaluator(
    labelCol="Booking_1", rawPredictionCol="indexedFeatures", metricName="areaUnderROC")
AUC = evaluator.evaluate(df_final1)
print(AUC)

0.5454063383547564


In [236]:
df_final1.sort(df_final1.Predict_R.asc()).select("Predict_Py","Predict_R","actual_conv","Traffic","Booking").show(100)

+--------------------+-------------------+-----------+-------+-------+
|          Predict_Py|          Predict_R|actual_conv|Traffic|Booking|
+--------------------+-------------------+-----------+-------+-------+
|0.004677447613073046|                0.0|        0.0|      4|      0|
|0.012561991107585588|                0.0|        0.0|      2|      0|
|0.005363802169741504|                0.0|        0.0|      2|      0|
|0.005779333072956203|                0.0|        0.0|      1|      0|
|0.005075028021679...|                0.0|        0.0|      2|      0|
|0.005566073383639957|                0.0|        0.0|      1|      0|
|0.005337270869345489|                0.0|        0.0|      1|      0|
|0.008580407728143082|2.56081946222791E-6|        0.0|      1|      0|
|0.005475832904084982|3.54609929078014E-6|        0.0|      4|      0|
|0.005098228108506...|3.64963503649635E-6|        0.0|      2|      0|
|0.006235755334705174|5.58659217877095E-6|        0.0|      2|      0|
|0.004